## Get the Wing-et SQLLite Database



In [ ]:
Invoke-WebRequest -Uri "https://winget.azureedge.net/cache/source.msix" -OutFile "$env:temp\Winget.zip"
Expand-Archive  "$env:temp\Winget.zip" -DestinationPath "$env:temp\Winget-Files" -Force
if (Test-path   "$env:temp\Winget-Files\Public\index.db") {
    Move-Item   "$env:temp\Winget-Files\Public\index.db" "$env:temp\Winget.db" -Force
    Remove-item "$env:temp\Winget-Files" -Recurse -Forc
    Remove-item "$env:temp\Winget.Zip" -Force
}

## Query the database to get the list of packages, their names, IDs versions, and manifest paths 

In [ ]:
$query = @"
    SELECT manifest.rowid , names.name, norm_names.norm_name, ids.id, versions.version, manifest.pathpart
    FROM   manifest 
    JOIN   ids            ON        ids.rowid = manifest.id  
    JOIN   names          ON      names.rowid = manifest.name 
    JOIN   versions       ON   versions.rowid = manifest.version 
    JOIN   norm_names_map ON   manifest.rowid = norm_names_map.manifest  
    JOIN   norm_names     ON norm_names.rowid = norm_names_map.norm_name
"@  # The SQLLite DB breaks the path to the manifest into parts which we will reasssemble 
    # first store the parts and their relationships in two hash tables.  

$pathparts = @{} 
$parents   = @{} 
Get-SQL -Lite  -Connection "$env:temp\Winget.db" -Table "pathparts" | ForEach-Object {
    $pathparts[$_.rowID] = $_.pathpart; 
    if ($_.parent) {$parents[$_.rowId] = $_.parent} 
} 

# Now run the query and use select-object to reassemble the the manifest path 
# Add a script method to get the manifest  -- can add more script methods to call winget to install / update / uninstall. 
# And then flag the latest version of each, and keep  Normalized names--> Manifest record, and ID --> versions hash tables.
$WingetData =  Get-SQL -Connection  "$env:temp\Winget.db" -Lite -SQL $query | 
    Select-object rowid, id, name, 
        @{n='normalizedName';e='norm_name'}, 
        @{n='VersionString'; e='version'},
        @{n='Version';       e={$_.version -as [System.Version]}},
        @{n='LatestVersion'; e={$false} },
        @{n="manifestURL";   e={
                                $path = $pathparts[$_.pathpart];  
                                $p    =   $parents[$_.pathpart]; 
                                while  ($p) {$path = $pathparts[$p] + "/" + $path; $p= $parents[$p]} 
                                "https://winget.azureedge.net/cache$path" }
        } | Add-Member -PassThru -MemberType ScriptMethod -Name "GetManifest" -Value {Invoke-RestMethod $this.manifestURL | ConvertFrom-Yaml} |
            Sort-Object -Property ID, Version

$nNamesHash = @{}
$WingetData | Where-Object {$_.version} |  
                Group-Object -Property ID | 
                    ForEach-Object  {  
                        $ver    = ($_.group | Measure-Object -Property version -Maximum ).Maximum
                        $newest =  $_.group.where({$_.version -eq $ver}) | Sort-Object -property VersionString | Select-object -Last 1
                        if ($newest) {
                            $newest.LatestVersion = $true
                            $nNamesHash[$newest.normalizedName] = $Newest
                        }
                } 

Write-Host "Found latest for $($nNamesHash.Count) packages"

$versionsHash = @{} 
foreach ($w in $WingetData) {
    if (-not $versionsHash[$w.id]) { 
          $versionsHash[$w.id]  =  @($w.VersionString) }
    else {$versionsHash[$w.id] +=    $w.VersionString  }
}

Get-Sql -Close
#delete file

41179 row(s) returned
17699 row(s) returned
InvalidOperation: 
Line |
  42 |                          $newest.LatestVersion = $true
     |                          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The property 'LatestVersion' cannot be found on this object. Verify that the property exists and can be set.
InvalidOperation: 
Line |
  42 |                          $newest.LatestVersion = $true
     |                          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The property 'LatestVersion' cannot be found on this object. Verify that the property exists and can be set.
InvalidOperation: 
Line |
  42 |                          $newest.LatestVersion = $true
     |                          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The property 'LatestVersion' cannot be found on this object. Verify that the property exists and can be set.
InvalidOperation: 
Line |
  42 |                          $newest.LatestVersion = $true
     |                          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | T

Error: System.OperationCanceledException: Command :SubmitCode: $query = @"
    SELECT manifest.rowid , names.nam ... cancelled.

## Some sample information 

In [ ]:
$r = $WingetData.where({$_.id -eq 'microsoft.edge' -and $_.latestVersion})
$r  | ft -a * | out-String -Width 240
$m = $r.getManifest() 
$m 


rowid id             name           normalizedName VersionString Version       LatestVersion manifestURL
----- --             ----           -------------- ------------- -------       ------------- -----------
23981 Microsoft.Edge Microsoft Edge microsoftedge  103.0.1264.44 103.0.1264.44          True https://winget.azureedge.net/cache/manifests/m/Microsoft/Edge/103.0.1264.44/8859-Microsoft.Edge.yaml



Name                           Value
----                           -----
Tags                           {blink, browser, chromium, edge…}
CopyrightUrl                   https://www.microsoft.com/en-us/servicesagreement/
Installers                     {System.Collections.Hashtable, System.Collections.Hashtable, System.Collections.Hashtable}
PackageVersion                 103.0.1264.44
ShortDescription               World-class performance with more privacy, more productivity, and more value while you browse.
Author                         Microsoft Corporation
Copyright                

In [ ]:
$m.Installers


Name                           Value
----                           -----
Architecture                   x64
InstallerUrl                   https://msedge.sf.dl.delivery.mp.microsoft.com/filestreamingservice/files/c96b75fd-157d-4494-9ffd-d299a0b9dd9d/MicrosoftEdgeEnterpriseX64.msi
ProductCode                    {E2726D8F-66EF-3612-A8E0-9AB41538D70D}
InstallerSha256                DC48DEA2F274BA61BC2FD0A378EE0A7AAD2545170671D6A46254C91CFB7C400A
Architecture                   x86
InstallerUrl                   https://msedge.sf.dl.delivery.mp.microsoft.com/filestreamingservice/files/13dd47aa-69df-4605-aaec-3874dcb0d1ae/MicrosoftEdgeEnterpriseX86.msi
ProductCode                    {46130127-F8E6-3C29-939B-59988ADA386F}
InstallerSha256                382F3B4748D3C417E147BA726C754583F5961C3C6F58C2BBA0979978760392F6
Architecture                   arm64
InstallerUrl                   https://msedge.sf.dl.delivery.mp.microsoft.com/filestreamingservice/files/a3c1fd07-2ce3-494d-9672-88863fdc901

In [ ]:
$versionshash.'Microsoft.AzureCLI' -join ", "

2.6.0, 2.7.0, 2.8.0, 2.10.1, 2.11.0, 2.11.1, 2.12.0, 2.12.1, 2.13.0, 2.14.0, 2.14.1, 2.14.2, 2.15.0, 2.15.1, 2.16.0, 2.17.0, 2.17.1, 2.18.0, 2.19.0, 2.19.1, 2.20.0, 2.21.0, 2.22.0, 2.22.1, 2.23.0, 2.24.0, 2.24.1, 2.25.0, 2.26.0, 2.26.1, 2.27.0, 2.27.1, 2.27.2, 2.28.0, 2.29.0, 2.29.1, 2.29.2, 2.30.0, 2.31.0, 2.32.0, 2.33.0, 2.33.1, 2.34.0, 2.34.1, 2.35.0, 2.36.0, 2.37.0


## Optionally Dump the data to Excel to play with it

In [ ]:
#Optional 
$wingetdata | Export-Excel

3336


In [ ]:
using namespace 'System.Management.Automation'
using namespace 'System.Management.Automation'
using namespace 'System.Management.Automation.Language'
using namespace 'System.Collections'
using namespace 'System.Collections.Generic'


In [ ]:

class OptopusLibVariableSetsCompleter  : System.Management.Automation.IArgumentCompleter{ # get library variable sets - we need to filter out scripts
    [System.Collections.IEnumerable[System.Management.Automation.CompletionResult]] CompleteArgument( [string]$CommandName, [string]$ParameterName, [string]$WordToComplete,
                                                      [System.Management.Automation.Language.CommandAst]$CommandAst, [System.Collections.IDictionary] $FakeBoundParameters) {
        $results        = [System.Collections.Generic.List[System.Management.Automation.CompletionResult]]::new()
        $wordToComplete = $wordToComplete -replace "^`"|^'|'$|`"$", ''
        (Invoke-OctopusMethod -EndPoint 'libraryvariablesets?contentType=Variables' -ExpandItems -Verbose:$false).
            where({$_.Name -like "$wordToComplete*"}).
                foreach({
                    if ($_.Name -Notmatch '\W'){$results.Add([System.Management.Automation.CompletionResult]::new(    $_.Name    , $_.Name, ([System.Management.Automation.CompletionResultType]::ParameterValue) , $_.Name)) }
                    else                       {$results.Add([System.Management.Automation.CompletionResult]::new("'$($_.Name)'" , $_.Name, ([System.Management.Automation.CompletionResultType]::ParameterValue) , $_.Name)) }
                })
        return $results
    }
}

To Do  Add classes for completing ID and version and merge with cobalt. Remove the need for Get-SQL 
